In [ ]:
import os, sys, time, random, torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm
import hyp
import models
import state_data as aug
import preprocess_data as prep


def store_result(store_epoch_acc_val, store_epoch_loss_val, store_qwk_epoch_loss_val, HIDDEN, ONE_HOT, DATA_AUG, NUM_FOLD, NUM_FOLDS, sub_training_dir):
    most_acc = max(store_epoch_acc_val)
    min_loss = min(store_epoch_loss_val)
    qwk_max_loss = max(store_qwk_epoch_loss_val)
    print("\nHighest accuracy of {} occured at {}...\nMinimum loss occured at {}... \nMaximum QWK metric of {} occured at {}".format(
        most_acc, store_epoch_acc_val.index(most_acc)+1, 
        store_epoch_loss_val.index(min_loss)+1, 
        qwk_max_loss, store_qwk_epoch_loss_val.index(qwk_max_loss)+1))
    with open(sub_training_dir+"/HYP.txt","w+") as f:
        f.write("EPOCH = {} \n".format(hyp.EPOCHS))
        f.write("LR = {} \n".format(hyp.LR))
        f.write("HIDDEN_LAYERS = {} \n".format(HIDDEN))
        f.write("ONE_HOT = {} \n".format(ONE_HOT))
        f.write("DATA_AUG = {} \n".format(DATA_AUG))
        f.write("NUM_FOLDS = {}/{} \n".format(NUM_FOLD, NUM_FOLDS))
        f.write("Highest accuracy of {} occured at {}...\nMinimum loss of {} occured at {}... \nMaximum QWK metric of {} occured at {}".format(
        most_acc, store_epoch_acc_val.index(most_acc)+1, 
        min_loss, store_epoch_loss_val.index(min_loss)+1, 
        qwk_max_loss, store_qwk_epoch_loss_val.index(qwk_max_loss)+1))
#     checkpoints = os.listdir(sub_training_dir)
#     for checkpoint in checkpoints:
#         if "checkpoint" in checkpoint:
#             checkpoint_num = int(checkpoint[checkpoint.index("_")+1:checkpoint.index(".")])
#             if checkpoint_num not in [store_qwk_epoch_loss_val.index(qwk_max_loss)+1,
#                                       store_epoch_loss_val.index(min_loss)+1,
#                                       store_epoch_acc_val.index(most_acc)+1]:
#                 os.remove(sub_training_dir+"/"+checkpoint)


def train_fold(model, HIDDEN, ONE_HOT, DATA_AUG, NUM_FOLD, NUM_FOLDS, data_train_loader, data_val_loader, sub_training_dir):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=hyp.LR, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    qwk_loss = cohen_kappa_score
    ce_loss = nn.CrossEntropyLoss().cuda()
    epoch = 0
    store_epoch_loss = []
    store_qwk_epoch_loss = []
    store_epoch_loss_val = []
    store_qwk_epoch_loss_val = []
    store_epoch_acc_val = []
    try:
        for e in tqdm(range(hyp.EPOCHS)):
            #scheduler.step()
            epoch = e + 1
            epoch_loss = 0
            qwk_epoch_loss = 0
            store_batch_loss = []
            store_qwk_batch_loss = []
            
            for batch_num, (X, y) in enumerate(data_train_loader):
                optimizer.zero_grad()
                prediction = model.forward(X.cuda())
                batch_loss = ce_loss(prediction, y)
                batch_loss.backward()
                qwk_batch_loss = qwk_loss(y.clone().detach().cpu().numpy(), 
                                          np.argmax(prediction.clone().detach().cpu().numpy(), axis=1), 
                                          weights="quadratic")
                optimizer.step()
                store_batch_loss.append(batch_loss.clone().cpu())
                store_qwk_batch_loss.append(qwk_batch_loss)
                epoch_loss = torch.FloatTensor(store_batch_loss).mean()
                qwk_epoch_loss = torch.FloatTensor(store_qwk_batch_loss).mean()
                
            store_epoch_loss.append(epoch_loss)
            store_qwk_epoch_loss.append(qwk_epoch_loss)
#             torch.save(model.state_dict(), "{}/checkpoint_{}.pth".format(sub_training_dir, epoch))

            model.eval()
            epoch_loss_val = 0
            qwk_epoch_loss_val = 0
            epoch_acc_val = 0
            store_batch_loss_val = []
            store_qwk_batch_loss_val = []
            store_batch_acc_val = []
            misclassified_images = []
            for batch_num, (X, y) in enumerate(data_val_loader):
                with torch.no_grad():
                    prediction = model.forward(X.cuda())
                batch_loss = ce_loss(prediction, y)
                qwk_batch_loss = qwk_loss(y.clone().detach().cpu().numpy(), 
                                          np.argmax(prediction.clone().detach().cpu().numpy(), axis=1), 
                                          weights="quadratic")
                misclassified = prediction.max(-1)[-1].squeeze().cpu() != y.cpu()
                misclassified_images.append(X[misclassified==1].cpu())
                batch_acc = misclassified.float().mean()
                store_batch_loss_val.append(batch_loss)
                store_qwk_batch_loss_val.append(qwk_batch_loss)
                store_batch_acc_val.append(batch_acc)
                epoch_loss_val = torch.FloatTensor(store_batch_loss_val).mean()
                qwk_epoch_loss_val = torch.FloatTensor(store_qwk_batch_loss_val).mean()
                epoch_acc_val = torch.FloatTensor(store_batch_acc_val).mean()
            store_epoch_loss_val.append(epoch_loss_val)
            store_qwk_epoch_loss_val.append(qwk_epoch_loss_val)
            store_epoch_acc_val.append(1-epoch_acc_val) # 1 - % of misclassified cases
            plt.plot(store_epoch_loss_val[1:], label="Validation Loss")
            plt.plot(store_qwk_epoch_loss_val[1:], label="Validation Metric(QWK)")
            plt.plot(store_epoch_acc_val[1:], label="Validation Accuracy")
            plt.legend()
            plt.grid()
            plt.savefig("{}/Loss.png".format(sub_training_dir))
            plt.close()
#             if len(misclassified_images) > 0:
#                 misclassified_images = np.concatenate(misclassified_images,axis=0)
#                 validation_dir = sub_training_dir+'/misclassified/checkpoint_{}'.format(epoch)
#                 os.mkdir(validation_dir)
            model.train()
        store_result(store_epoch_acc_val, store_epoch_loss_val, store_qwk_epoch_loss_val, HIDDEN, ONE_HOT, DATA_AUG, NUM_FOLD, NUM_FOLDS, sub_training_dir)
        return store_qwk_epoch_loss_val, store_epoch_loss_val, store_epoch_acc_val
    except KeyboardInterrupt:
        store_result(store_epoch_acc_val, store_epoch_loss_val, store_qwk_epoch_loss_val, HIDDEN, ONE_HOT, DATA_AUG, NUM_FOLD, NUM_FOLDS, sub_training_dir)
        return store_qwk_epoch_loss_val, store_epoch_loss_val, store_epoch_acc_val


def train(HIDDEN_LIST, ONE_HOT, DATA_AUG, NUM_FOLDS, data_loader):
    print("Training..."+"HIDDEN_LIST/ONE_HOT/DATA_AUG/NUM_FOLDS", str(HIDDEN_LIST), ONE_HOT, DATA_AUG, NUM_FOLDS)
    training_dir = './training_{}_{}+{}+{}+{}'.format(int(time.time()), ONE_HOT, DATA_AUG, len(HIDDEN_LIST), max(HIDDEN_LIST))
    os.mkdir(training_dir)

    qwk_loss = [None] * NUM_FOLDS
    ce_loss = [None] * NUM_FOLDS
    acc = [None] * NUM_FOLDS
    for NUM_FOLD in range(NUM_FOLDS):
        sub_training_dir = training_dir+'/'+str(NUM_FOLD)
        os.mkdir(sub_training_dir)
#         os.mkdir(sub_training_dir+'/misclassified')
        data_train_loader, data_val_loader = prep.fold_divider(data_loader, NUM_FOLD, NUM_FOLDS)
        qwk_loss[NUM_FOLD], ce_loss[NUM_FOLD], acc[NUM_FOLD] = train_fold(models.Model(HIDDEN_LIST, ONE_HOT, DATA_AUG).cuda(), HIDDEN_LIST, ONE_HOT, DATA_AUG, NUM_FOLD, NUM_FOLDS, data_train_loader, data_val_loader, sub_training_dir)        

    avg_qwk_loss = torch.tensor(qwk_loss).mean(0).numpy().tolist()
    avg_ce_loss = torch.tensor(ce_loss).mean(0).numpy().tolist()
    avg_acc = torch.tensor(acc).mean(0).numpy().tolist()

    plt.plot(avg_ce_loss[1:], label="Validation Loss")
    plt.plot(avg_qwk_loss[1:], label="Validation Metric(QWK)")
    plt.plot(avg_acc[1:], label="Validation Accuracy")
    plt.legend()
    plt.grid()
    plt.savefig("{}/Avg_Loss.png".format(training_dir))
    plt.close()

    most_acc = max(avg_acc)
    min_loss = min(avg_ce_loss)
    qwk_max_loss = max(avg_qwk_loss)
    with open(training_dir+"/HYP.txt","w+") as f:
        f.write("EPOCH = {} \n".format(hyp.EPOCHS))
        f.write("LR = {} \n".format(hyp.LR))
        f.write("HIDDEN_LAYERS = {} \n".format(HIDDEN_LIST))
        f.write("ONE_HOT = {} \n".format(ONE_HOT))
        f.write("DATA_AUG = {} \n".format(DATA_AUG))
        f.write("Highest avg accuracy of {} occured at {}...\nMinimum avg loss of {} occured at {}... \nMaximum avg QWK metric of {} occured at {}".format(
        most_acc, avg_acc.index(most_acc)+1, 
        min_loss, avg_ce_loss.index(min_loss)+1, 
        qwk_max_loss, avg_qwk_loss.index(qwk_max_loss)+1))


if __name__ == "__main__":
    for DATA_AUG in [0, 1]:
        ONE_HOT = 1
        HIDDEN_LIST = hyp.HIDDEN_LIST[1]
        NUM_FOLDS = 10
        data_loader = prep.preprocess_data(ONE_HOT, DATA_AUG)
        train(HIDDEN_LIST, ONE_HOT, DATA_AUG, NUM_FOLDS, data_loader)

Data preparing...


  0%|          | 0/1000 [00:00<?, ?it/s]

Data prepared.
Training...HIDDEN_LIST/ONE_HOT/DATA_AUG/NUM_FOLDS [250, 500, 500, 250] 1 0 10


 22%|██▏       | 223/1000 [08:22<28:50,  2.23s/it]

In [5]:
import pandas as pd
import numpy as np
import torch, random, models
import torch.utils.data as data
import state_data as aug
import pandas as pd
import numpy as np
import torch, random, models, os, json
import torch.utils.data as data
import state_data as aug
import preprocess_data as prep

df = pd.read_csv('data/train.csv')

# df = prep.add_sentiment_data(df)

df = df.drop(['Name', 'RescuerID', 'PetID', 'Description'], axis=1)

In [6]:
d = torch.FloatTensor(df.values)

In [15]:
a = d[:,8:10]

In [16]:
type(a)

torch.Tensor

In [17]:
a.shape

torch.Size([14993, 2])

In [184]:
b = torch.FloatTensor(nType)

In [164]:
type(b)

torch.Tensor

In [165]:
b

tensor([[1., 0.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [1., 0.],
        [0., 1.]])

In [161]:
b.shape

torch.Size([14993, 2])

In [159]:
Y = torch.cat([a,b], dim=0).cuda() # Age

RuntimeError: invalid argument 0: Tensors must have same number of dimensions: got 1 and 2 at /opt/conda/conda-bld/pytorch_1549628766161/work/aten/src/TH/generic/THTensorMoreMath.cpp:1298